# Build a conversational agent with tensorflow.

Based on: 
    - A Neural Conversational Model https://arxiv.org/abs/1506.05869
    - http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf 
    - https://www.tensorflow.org/versions/r0.11/tutorials/seq2seq/index.html
Steps:
    1. Parse XML files
    2. Build model
    3. Test model

## Data: Movie subtitles

Download from: http://opus.lingfil.uu.se/OpenSubtitles.php

Extract with:
    1. tar -xvf es.tar.gz
    2. find . -name '*.gz' -exec gunzip '{}' \;
    

## 01 Parse XML files

Execute once

In [1]:
from __future__ import print_function

data_path = '/home/jorge/data/training/text/' 



In [2]:
# Generator of list of files in a folder and subfolders
import os
import shutil
import fnmatch

def gen_find(filepattern, toppath):
    '''
    Generator with a recursive list of files in the toppath that match filepattern 
    Inputs:
        filepattern(str): Command stype pattern 
        toppath(str): Root path
    '''
    for path, dirlist, filelist in os.walk(toppath):
        for name in fnmatch.filter(filelist, filepattern):
            yield os.path.join(path, name)

#Test
print(gen_find("*.xml", data_path+'OpenSubtitles/').next())

/home/jorge/data/training/text/OpenSubtitles/es/Drama/1979/6341_99961_136507_quadrophenia.xml


In [3]:
def cleanText(text):
    t = text
    t = t.replace('...', ' ')
    t = t.replace('... ...', ' ')
    t = t.replace(' .. ..', ' ')
    t = t.replace('  ', ' ')
    #t = text.strip('-')
    t = t.lower()
    t = t.replace('-', '')
    #t = t.strip('\"')
    #regex = re.compile('\(.+?\)')
    #t = regex.sub('', t)
    #t.replace('  ', ' ')
    #regex = re.compile('\{.+?\}')
    #t = regex.sub('', t)
    #t = t.replace("~", "")
    #t = t.strip(' ')
    if t[-1]=='.':
        t = t[:-1]
    return t

print(cleanText('a-a .. .. Aa...  a.'))

aa aa  a


In [4]:
import xml.etree.ElementTree as ET


def extractTokenizedPhrases(xmlFilePath, dataDirFilePath):
    '''
    Extract raw phrases from the xml subtitles files:
    Based on: https://github.com/inikdom/opensubtitles-parser/blob/master/opensubtitleparser.py 
    '''
    global inc
    inc += 1
    tree = ET.parse(xmlFilePath)
    root = tree.getroot()
    print("Processing " + xmlFilePath + "...")
    for child in root.findall('s'):
        A = []
        for node in child.getiterator():
            if node.tag == 'w':
                    A.append(node.text.encode('utf-8', 'ignore'))
        text = " ".join(A)
        text = cleanText(text)
        try:
            if text[0] != '[' and text[-1] != ':':
                with open(dataDirFilePath + str(inc).zfill(4) +'raw.txt', 'a') as f:
                    f.write(text + "\n")
        except IndexError:
            pass

In [5]:
subtitles_list= gen_find("*1.xml", data_path+'OpenSubtitles/')
for i in subtitles_list:
    print(i)

/home/jorge/data/training/text/OpenSubtitles/es/Sci-Fi/1997/415_97562_132818_stargate_sg-1.xml
/home/jorge/data/training/text/OpenSubtitles/es/Thriller/2004/7752_78938_104474_1111.xml
/home/jorge/data/training/text/OpenSubtitles/es/Action/2003/149_117044_159130_kill_bill_vol_1.xml
/home/jorge/data/training/text/OpenSubtitles/es/Action/2003/149_128721_177075_kill_bill_vol_1.xml
/home/jorge/data/training/text/OpenSubtitles/es/Action/2003/149_56103_71213_kill_bill_vol_1.xml
/home/jorge/data/training/text/OpenSubtitles/es/Documentary/2004/142_80382_106697_fahrenheit_911.xml
/home/jorge/data/training/text/OpenSubtitles/es/Documentary/2004/142_80382_106696_fahrenheit_911.xml
/home/jorge/data/training/text/OpenSubtitles/es/Crime/2000/3046_74528_97624_gangster_no_1.xml


In [6]:
#Parser for the XML files

inc = 0

subtitles_list= gen_find("*.xml", data_path+'OpenSubtitles/')

processed_data_dir = data_path + 'OpenSubtitlesRaw/'
if not os.path.exists(processed_data_dir):
    os.makedirs(processed_data_dir)

for f in subtitles_list:
    try:
        extractTokenizedPhrases(f, processed_data_dir)
    except Exception as e:
        print(e)
        print("Error in " + f)
        pass

print('Files processed: ',inc)


Processing /home/jorge/data/training/text/OpenSubtitles/es/Drama/1979/6341_99961_136507_quadrophenia.xml...
Processing /home/jorge/data/training/text/OpenSubtitles/es/Drama/1985/975_65902_84973_ran.xml...
Processing /home/jorge/data/training/text/OpenSubtitles/es/Drama/1992/1141_224295_296807_twin_peaks_fire_walk_with_me.xml...
Processing /home/jorge/data/training/text/OpenSubtitles/es/Drama/1992/850_44047_54941_dracula.xml...
Processing /home/jorge/data/training/text/OpenSubtitles/es/Drama/1992/850_152169_207480_dracula.xml...
Processing /home/jorge/data/training/text/OpenSubtitles/es/Drama/1995/321_69448_90069_before_sunrise.xml...
Processing /home/jorge/data/training/text/OpenSubtitles/es/Drama/1995/1625_61484_78893_twelve_monkeys.xml...
Processing /home/jorge/data/training/text/OpenSubtitles/es/Drama/1951/1151_79222_104883_a_streetcar_named_desire.xml...
Processing /home/jorge/data/training/text/OpenSubtitles/es/Drama/2000/2049_108409_148207_wonder_boys.xml...
Processing /home/jorg

# Read sentences and generate data structures

In [3]:
from __future__ import print_function

data_path = '/home/jorge/data/training/text/' 



In [4]:
# Process raw files to convert in sequences

# Separate train/test by movie
train_files = gen_find("*[1-9]raw.txt", data_path+'OpenSubtitlesRaw/')
test_files = gen_find("*0raw.txt", data_path+'OpenSubtitlesRaw/')

#Read train files and create dictionary



In [5]:
def read_sentences(generator):
    sentences = []
    for ff in generator:
        with open(ff, 'r') as f:
            sentences += f.read().splitlines()
    return sentences        

#Test
train_sentences = read_sentences(train_files)
print(len(train_sentences))
print(train_sentences[:10])

1883869
['algunos creen  que la vida aqui comenzo alli fuera ,', 'lejos en el universo , con tribus de humanos  quienes podrian haber sido los antepasados de los egipcios  o los toltecs  o los mayas  que fueron los grandes arquitectos de las grandes piramides  o la civilisacion perdida de lemuria  o atlantida . algunos creen que pueden existir todavia hermanos humanos  que hasta hoy luchan para sobrevivir ,', 'lejos , lejos entre las estrellas ', 'nobles delegados , supongo que estan deseosos de volver a sus naves  antes de nuestro encuentro con los cylons , pero creo que es apropiado brindar  por el mas significante acontecimiento en la historia de la humanidad ', 'me gustaria elevar mi chalis  a ustedes  no solo como el quorum de 12 , representando las 12 colonias de hombres pero como mis amigos ', 'los mayores lideres nunca antes reunidos . mientras nos acercamos al septimo milenio del tiempo', 'la raza humana al fin encontrara paz , gracias a ustedes ', 'paz !', "starbuck , el vien

In [6]:
test_sentences = read_sentences(test_files)

In [7]:
from nltk import word_tokenize

def tokenize(sentences):
    
    print('Tokenizing...')
    tokens = []
    for sentence in sentences:
        tokens += [word_tokenize(sentence.decode('utf-8'), language='spanish')]
    print('Done!')

    return tokens

train_tokens = tokenize(train_sentences)
print(train_tokens[:100])

Tokenizing...
Done!
[[u'algunos', u'creen', u'que', u'la', u'vida', u'aqui', u'comenzo', u'alli', u'fuera', u','], [u'lejos', u'en', u'el', u'universo', u',', u'con', u'tribus', u'de', u'humanos', u'quienes', u'podrian', u'haber', u'sido', u'los', u'antepasados', u'de', u'los', u'egipcios', u'o', u'los', u'toltecs', u'o', u'los', u'mayas', u'que', u'fueron', u'los', u'grandes', u'arquitectos', u'de', u'las', u'grandes', u'piramides', u'o', u'la', u'civilisacion', u'perdida', u'de', u'lemuria', u'o', u'atlantida', u'.', u'algunos', u'creen', u'que', u'pueden', u'existir', u'todavia', u'hermanos', u'humanos', u'que', u'hasta', u'hoy', u'luchan', u'para', u'sobrevivir', u','], [u'lejos', u',', u'lejos', u'entre', u'las', u'estrellas'], [u'nobles', u'delegados', u',', u'supongo', u'que', u'estan', u'deseosos', u'de', u'volver', u'a', u'sus', u'naves', u'antes', u'de', u'nuestro', u'encuentro', u'con', u'los', u'cylons', u',', u'pero', u'creo', u'que', u'es', u'apropiado', u'brindar', u'por

In [8]:
test_tokens = tokenize(test_sentences)

Tokenizing...
Done!


In [9]:
import numpy as np

#create the dictionary to conver words to numbers. Order it with most frequent words first
def build_dict(sentences):
    '''
    Build dictionary of train words
    Outputs: 
     - Dictionary of word --> word index
     - Dictionary of word --> word count freq
    '''
    print('Building dictionary..',)
    wordcount = dict()
    #For each worn in each sentence, cummulate frequency
    for ss in sentences:
        for w in ss:
            if w not in wordcount:
                wordcount[w] = 1
            else:
                wordcount[w] += 1

    counts = wordcount.values() # List of frequencies
    keys = wordcount.keys() #List of words
    
    sorted_idx = reversed(np.argsort(counts))
    
    worddict = dict()
    for idx, ss in enumerate(sorted_idx):
        worddict[keys[ss]] = idx+4  # leave 0-> PAD, 1-> GO, 2->EOL, 3->UNK
    print(np.sum(counts), ' total words ', len(keys), ' unique words')

    return worddict, wordcount


worddict, wordcount = build_dict(train_tokens)



Building dictionary..
11825024  total words  151886  unique words


In [10]:
print(worddict['la'], wordcount['la'])

12 207604


## Encode sentences and build the data structures
    - The target sentence is the next to the input sentence
    - Only sentences with input and target length < 15
        - input sentence: [PAD, PAD, PAD, PAD; A, UWN, C, D]
        - Target sentence: [GO, X, Y, UWN, EOF, PAD, PAD]
        - Weigth sentence: [ 1, 1, 1,   1,   1,   0,   0]

In [11]:
seq_length = 5
vocab_size = 10000

PAD_ID = 0
GO_ID = 1
EOL_ID = 2
UNK_ID = 3

def worddict_id(w):
    return worddict[w] if( w in worddict and worddict[w] < vocab_size) else UNK_ID

def create_sequences(tokens):
    input_sent = []
    target_sent = []

    for i in range(1,len(tokens[1:])):
        sent_i = tokens[i-1]
        sent_t = tokens[i]
        if len(sent_t) < seq_length and len(sent_i) < seq_length:
            input_s = [PAD_ID]*(seq_length - len(sent_i)) + [worddict_id(w) for w in sent_i]
            target_s = [GO_ID] + [worddict_id(w) for w in sent_t] + [EOL_ID] + [PAD_ID]*(seq_length-len(sent_t))
            input_sent += [input_s]
            target_sent += [target_s]
    return np.array(input_sent, dtype=np.int32), np.array(target_sent, dtype=np.int32)


In [12]:
input_sent_trn, target_sent_trn = create_sequences(train_tokens)
print(input_sent_trn.shape)
print(target_sent_trn.shape)


(386509, 5)
(386509, 7)


In [13]:
input_sent_tst, target_sent_tst = create_sequences(test_tokens)
print(input_sent_tst.shape)
print(target_sent_tst.shape)


(43041, 5)
(43041, 7)


In [14]:
print(test_sentences[:4])
print(test_tokens[:4])
print(input_sent_tst[:4])
print(target_sent_tst[:4])

['les contar\xc3\xa9 un chiste viejo ', '" hay que ver lo mala que es aqu\xc3\xad la comida " ', 'y la otra dice " ya , y adem\xc3\xa1s las raciones son muy peque\xc3\xb1as " ', 'as\xc3\xad es c\xc3\xb3mo veo yo la vida ']
[[u'les', u'contar\xe9', u'un', u'chiste', u'viejo'], [u'``', u'hay', u'que', u'ver', u'lo', u'mala', u'que', u'es', u'aqu\xed', u'la', u'comida', u'``'], [u'y', u'la', u'otra', u'dice', u'``', u'ya', u',', u'y', u'adem\xe1s', u'las', u'raciones', u'son', u'muy', u'peque\xf1as', u'``'], [u'as\xed', u'es', u'c\xf3mo', u'veo', u'yo', u'la', u'vida']]
[[   0    0    3   45  457]
 [   0    0    0    0 1508]
 [   0    9   22 3274 1508]
 [   0   55    3    4    3]]
[[   1    3    2    0    0    0    0]
 [   1    9   22 3274 1508    2    0]
 [   1    8   21   23    5    2    0]
 [   1  422    4 1508    2    0    0]]


In [15]:
#Save data 
np.save('/tmp/input_sent_trn',  input_sent_trn )
np.save('/tmp/target_sent_trn', target_sent_trn)
np.save('/tmp/input_sent_tst',  input_sent_tst )
np.save('/tmp/target_sent_tst', target_sent_tst)

#Save worddict
import pickle
pickle.dump( worddict, open( "/tmp/worddict.p", "wb" ) )



## 02 Build model

In [1]:
import tensorflow as tf
import numpy as np
import random

seq_length = 5
vocab_size = 10000

PAD_ID = 0
GO_ID = 1
EOL_ID = 2
UNK_ID = 3


input_sent_trn  = np.load('/tmp/input_sent_trn.npy' )
target_sent_trn = np.load('/tmp/target_sent_trn.npy')
input_sent_tst  = np.load('/tmp/input_sent_tst.npy' )
target_sent_tst = np.load('/tmp/target_sent_tst.npy')

print(input_sent_trn.shape)

import pickle
worddict = pickle.load( open( "/tmp/worddict.p", "rb" ) )


(386509, 5)


In [2]:
#Start interactive session
#sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
#sess = tf.InteractiveSession()

ops.reset_default_graph()
try:
    sess.close()
except:
    
    pass

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))



In [3]:
#Define the model

batch_size = 32
vocab_size = vocab_size + 4
embedding_dim = 300
memory_dim = 1024
num_layers = 3

#Placeholders
enc_inp = [tf.placeholder(tf.int32, shape=(None,), name="inp%i" % t) for t in range(seq_length)]

dec_inp = [tf.placeholder(tf.int32, shape=(None,), name="labels%i" % t) for t in range(seq_length+2)]

weights = [tf.placeholder(tf.float32, shape=(None,), name="weights%i" % t) for t in range(seq_length+2)]

# Our targets are decoder inputs shifted by one.
targets = [dec_inp[i + 1]  for i in range(len(dec_inp) - 1)] + [dec_inp[-1]]
print('targets', targets)

learning_rate = tf.placeholder(tf.float32, name='learning_rate')
feed_previous = tf.placeholder(tf.bool, name='feed_previous')


#Model

# Initial memory value for recurrence.
#prev_mem = tf.zeros((batch_size, memory_dim)) #???



# Decode sampled_loss
#output_projection = None
w = tf.get_variable("proj_w", [memory_dim, vocab_size])
#W_proy = tf.transpose(w)
b_proy = tf.get_variable("proj_b", [vocab_size])
output_projection = (w,b_proy)


# Recurrent layers
single_cell = tf.nn.rnn_cell.GRUCell(memory_dim)
#single_cell = tf.nn.rnn_cell.BasicLSTMCell(memory_dim)
cell = tf.nn.rnn_cell.MultiRNNCell([single_cell] * num_layers)

dec_outputs, dec_memory = tf.nn.seq2seq.embedding_attention_seq2seq(
    enc_inp, dec_inp, cell, vocab_size, vocab_size, embedding_dim, 
    output_projection = output_projection,
    feed_previous=feed_previous,
    scope='emb')
print('dec_outputs',dec_outputs)

#For score: Convert output to the original softmax values
if output_projection is not None:
    outputs = [tf.matmul(d_o, output_projection[0]) + output_projection[1] for d_o in dec_outputs]
else:
    outputs = dec_outputs
print('outputs',outputs)
    
    
#Loss
num_samples = 32
def sampled_loss(inputs, targets):
    targets = tf.reshape(targets, [-1, 1])
    localWt = tf.cast(tf.transpose(w), tf.float32)
    localB  = tf.cast(b, tf.float32)
    localInputs = tf.cast(inputs, tf.float32)
    
    return tf.cast(
        tf.nn.sampled_softmax_loss(localWt, localB, localInputs, targets, num_samples, vocab_size),
        tf.float32)






loss = tf.nn.seq2seq.sequence_loss(dec_outputs, targets, weights, softmax_loss_function=sampled_loss, name='seq2seq')

#Trainner
#learning_rate = 0.5
#momentum = 0.9
optimizer = tf.train.AdamOptimizer(learning_rate,)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)





('targets', [<tf.Tensor 'labels1:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels2:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels3:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels4:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels5:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels6:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels6:0' shape=(?,) dtype=int32>])


ValueError: fn1 and fn2 must return the same number of results.

In [3]:
#Define the model V02

batch_size = 32
vocab_size = vocab_size + 4
embedding_dim = 300
memory_dim = 1024
num_layers = 3

#Placeholders
enc_inp = [tf.placeholder(tf.int32, shape=(None,), name="inp%i" % t) for t in range(seq_length)]

dec_inp = [tf.placeholder(tf.int32, shape=(None,), name="labels%i" % t) for t in range(seq_length+2)]

weights = [tf.placeholder(tf.float32, shape=(None,), name="weights%i" % t) for t in range(seq_length+2)]

# Our targets are decoder inputs shifted by one.
targets = [dec_inp[i + 1]  for i in range(len(dec_inp) - 1)] + [dec_inp[-1]]
print('targets', targets)

learning_rate = tf.placeholder(tf.float32, name='learning_rate')
feed_previous = tf.placeholder(tf.bool)


#Model

# Initial memory value for recurrence.
#prev_mem = tf.zeros((batch_size, memory_dim)) #???



# Decode sampled_loss
w_proy = tf.get_variable("proj_w", [memory_dim, vocab_size])
b_proy = tf.get_variable("proj_b", [vocab_size])



# Recurrent layers
single_cell = tf.nn.rnn_cell.GRUCell(memory_dim)
#single_cell = tf.nn.rnn_cell.BasicLSTMCell(memory_dim)
cell = tf.nn.rnn_cell.MultiRNNCell([single_cell] * num_layers)



#For score: Convert output to the original softmax values
def outputs_train():
    dec_outputs, dec_memory = tf.nn.seq2seq.embedding_attention_seq2seq(
                enc_inp, dec_inp, cell, vocab_size, vocab_size, embedding_dim, 
                output_projection = (w_proy, b_proy),
                feed_previous=feed_previous,
                scope='emb1')
    outputs = [tf.matmul(d_o, w_proy) + b_proy for d_o in dec_outputs]
    return dec_outputs, outputs
    
def outputs_score():
    dec_outputs, dec_memory = tf.nn.seq2seq.embedding_attention_seq2seq(
                enc_inp, dec_inp, cell, vocab_size, vocab_size, embedding_dim, 
                output_projection = None,
                feed_previous=feed_previous,
                scope='emb2')
    return dec_outputs, dec_outputs

outputs, dec_outputs = tf.cond(feed_previous, outputs_train, outputs_score)

print('dec_outputs', dec_outputs)
print('outputs', outputs)


    

loss = tf.nn.seq2seq.sequence_loss(dec_outputs, targets, weights, softmax_loss_function=sampled_loss, name='seq2seq')

#Trainner
#learning_rate = 0.5
#momentum = 0.9
optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-08)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)


saver = tf.train.Saver(max_to_keep=0)




('targets', [<tf.Tensor 'labels1:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels2:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels3:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels4:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels5:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels6:0' shape=(?,) dtype=int32>, <tf.Tensor 'labels6:0' shape=(?,) dtype=int32>])


AttributeError: 'list' object has no attribute 'name'

In [4]:
#Start interactive session
sess.run(tf.initialize_all_variables())

In [34]:
def worddict_id(w):
    return worddict[w] if( w in worddict and worddict[w] < vocab_size) else UNK_ID


#Inverse dictionary
inv_worddict = {v: k for k, v in worddict.iteritems()}
inv_worddict[0] = 'PAD'
inv_worddict[1] = 'GO'
inv_worddict[2] = 'EOL'
inv_worddict[3] = 'UNK'

def decode_response(response_array):
    response_text = []
    for i in range(response_array.shape[1]):
        response_dec = [np.argmax(r) for r in response_array[:,i,:]]
        response_text += [' '.join([inv_worddict[r] for r in response_dec])]
    return response_text

12078


In [5]:
def train_batch(n_epoch=1, batch_size, lr=0.001):
    
    for epoch in range(n_epoch):
        loss_cumm = []
        #for i, init_pos in enumerate(range(0, input_sent_trn.shape[0], batch_size)):
        for i, init_pos in enumerate(range(0, 1000, batch_size)):
        
            encoder_input = input_sent_trn[init_pos:init_pos+batch_size]
            decoder_input = target_sent_trn[init_pos:init_pos+batch_size]
            W = [[1]*(np.where(y==EOL_ID)[0][0]+1) + [0]*(seq_length - np.where(y==EOL_ID)[0][0]+1) for y in decoder_input]

            # Dimshuffle to seq_len * batch_size
            encoder_input = np.array(encoder_input).T
            decoder_input = np.array(decoder_input).T
            W = np.array(W).T

            feed_dict = {learning_rate: lr, feed_previous: False} # False feed_previous in the trainig process.
            feed_dict.update({enc_inp[t]: encoder_input[t] for t in range(seq_length)})
            feed_dict.update({dec_inp[t]: decoder_input[t] for t in range(seq_length+2)})
            feed_dict.update({weights[t]: W[t] for t in range(seq_length+2)})

            _, loss_t = sess.run([train_op, loss], feed_dict)
            loss_cumm += [loss_t]
            
            if i%10 == 0:
                print(epoch, i, loss_t, loss_cumm)
        #Test
        loss_cumm_tst = []
        #for i, init_pos in enumerate(range(0, input_sent_tst.shape[0], batch_size)):
        for i, init_pos in enumerate(range(0, 100, batch_size)):

            encoder_input_tst = input_sent_tst[init_pos:init_pos+batch_size]
            decoder_input_tst = target_sent_tst[init_pos:init_pos+batch_size]
            W_tst = [[1]*(np.where(y==EOL_ID)[0][0]+1) + [0]*(seq_length - np.where(y==EOL_ID)[0][0]+1) for y in decoder_input_tst]

            # Dimshuffle to seq_len * batch_size
            encoder_input_tst = np.array(encoder_input_tst).T
            decoder_input_tst = np.array(decoder_input_tst).T
            W_tst = np.array(W_tst).T

            feed_dict = {feed_previous: False} # False feed_previous in the test process.
            feed_dict.update({enc_inp[t]: encoder_input_tst[t] for t in range(seq_length)})
            feed_dict.update({dec_inp[t]: decoder_input_tst[t] for t in range(seq_length+2)})
            feed_dict.update({weights[t]: W_tst[t] for t in range(seq_length+2)})

            loss_tst = sess.run(loss, feed_dict)
            loss_cumm_tst += [loss_tst]
            
            #Evaluate the output of a batch
            if i==0:
                feed_dict = {feed_previous: True} # True feed_previous in the score process.
                feed_dict.update({enc_inp[t]: encoder_input_tst[t] for t in range(seq_length)})

                response_predict = np.array(sess.run(outputs, feed_dict) )
                response_predict_text = decode_response(response_predict)
                input_real_text = [' '.join([inv_worddict[r] for r in rr]) for rr in encoder_input_tst]


        print(epoch, i, loss_cumm, loss_cumm_tst)
        for i in range(len(input_real_text)):
            print(input_real_text[i], ' - ',response_predict_text[i])
                   
        save_path = saver.save(sess, path + "/tmp/seq2seq_model/saver_"+str(epoch))
        print("Model saved in file: %s" % save_path)
        
    return loss_t



In [6]:
loss_cumm=[]
lr=0.001
for t in range(20000):
    loss_t = train_batch(batch_size, lr=lr)
    loss_cumm += [loss_t]
    if t>=100 and t%100==0:
        print(t, lr, loss_t, np.mean(loss_cumm[-100:]))
    #Update lr if not improvements in last 50 batches
    #if t>100 and np.mean(loss_cumm[t-100:t-50])<np.mean(loss_cumm[t-50:]):  
    #    lr = lr*0.9999


(100, 0.001, 2.6680968, 3.3680258)
(200, 0.001, 1.6797605, 2.4738441)
(300, 0.001, 1.9919605, 2.2371612)
(400, 0.001, 1.9533697, 2.0596545)
(500, 0.001, 1.9357147, 1.9423594)
(600, 0.001, 1.7343981, 1.9004714)
(700, 0.001, 1.4882658, 1.8825679)
(800, 0.001, 1.7425336, 1.8023398)
(900, 0.001, 1.3208145, 1.76546)
(1000, 0.001, 1.7532952, 1.7672626)
(1100, 0.001, 1.8037391, 1.7160783)
(1200, 0.001, 1.6614611, 1.7353935)
(1300, 0.001, 1.4938455, 1.6753579)
(1400, 0.001, 1.8835199, 1.6851568)
(1500, 0.001, 1.4241931, 1.6706657)
(1600, 0.001, 1.6986302, 1.6580102)
(1700, 0.001, 1.8536155, 1.6686645)
(1800, 0.001, 1.3667629, 1.6226645)
(1900, 0.001, 1.6591893, 1.5877286)
(2000, 0.001, 1.4565079, 1.6163501)
(2100, 0.001, 1.4917675, 1.6028264)
(2200, 0.001, 1.5958592, 1.574126)
(2300, 0.001, 1.5764931, 1.5808281)
(2400, 0.001, 1.3433313, 1.5401301)
(2500, 0.001, 1.4022131, 1.5418406)
(2600, 0.001, 1.3931156, 1.560645)
(2700, 0.001, 1.7195032, 1.5533414)
(2800, 0.001, 1.3522923, 1.538218)
(2900,

In [9]:
loss_cumm=[]
lr=0.001
for t in range(20000):
    loss_t = train_batch(batch_size, lr=lr)
    loss_cumm += [loss_t]
    if t>=100 and t%100==0:
        print(t, lr, loss_t, np.mean(loss_cumm[-100:]))
    #Update lr if not improvements in last 50 batches
    #if t>100 and np.mean(loss_cumm[t-100:t-50])<np.mean(loss_cumm[t-50:]):  
    #    lr = lr*0.9999

(100, 0.001, 1.5130278, 1.2000957)
(200, 0.001, 1.0776401, 1.2553208)
(300, 0.001, 0.96079272, 1.2184376)
(400, 0.001, 1.1673901, 1.1813493)
(500, 0.001, 0.93296045, 1.2149066)
(600, 0.001, 1.1656467, 1.1860203)
(700, 0.001, 1.5785084, 1.2031512)
(800, 0.001, 1.5961406, 1.204002)
(900, 0.001, 1.1843152, 1.1684541)
(1000, 0.001, 1.0226066, 1.2089456)
(1100, 0.001, 1.0788701, 1.2168354)
(1200, 0.001, 1.2275218, 1.1658794)
(1300, 0.001, 1.0283326, 1.1673231)
(1400, 0.001, 1.417272, 1.2051666)
(1500, 0.001, 1.2479377, 1.1646857)
(1600, 0.001, 1.5664592, 1.2159858)
(1700, 0.001, 1.2183909, 1.1944628)
(1800, 0.001, 1.208905, 1.2223117)
(1900, 0.001, 1.5202186, 1.1775222)
(2000, 0.001, 0.80744267, 1.1919824)
(2100, 0.001, 1.0342281, 1.1668514)
(2200, 0.001, 1.0938537, 1.206875)
(2300, 0.001, 1.0527415, 1.222261)
(2400, 0.001, 1.4480567, 1.2122655)
(2500, 0.001, 1.1699696, 1.2073081)
(2600, 0.001, 0.90641356, 1.194724)
(2700, 0.001, 1.0456498, 1.2208441)
(2800, 0.001, 1.0226095, 1.1831585)
(29

In [12]:
loss_cumm=[]
lr=0.001
for t in range(20000):
    loss_t = train_batch(batch_size, lr=lr)
    loss_cumm += [loss_t]
    if t>=100 and t%100==0:
        print(t, lr, loss_t, np.mean(loss_cumm[-100:]))
    #Update lr if not improvements in last 50 batches
    #if t>100 and np.mean(loss_cumm[t-100:t-50])<np.mean(loss_cumm[t-50:]):  
    #    lr = lr*0.9999

(100, 0.001, 1.2567699, 1.1142534)
(200, 0.001, 1.1602849, 1.1074492)
(300, 0.001, 0.99159437, 1.1098237)
(400, 0.001, 0.9649092, 1.1025397)
(500, 0.001, 1.0334984, 1.1101135)
(600, 0.001, 1.322319, 1.1131977)
(700, 0.001, 0.80184078, 1.1210097)
(800, 0.001, 1.3315003, 1.1334952)
(900, 0.001, 1.1221209, 1.0958219)
(1000, 0.001, 1.2846539, 1.1134217)
(1100, 0.001, 1.1094085, 1.1015562)
(1200, 0.001, 1.1443317, 1.1134106)
(1300, 0.001, 1.1210375, 1.0753226)
(1400, 0.001, 0.97469765, 1.114676)
(1500, 0.001, 1.0402653, 1.1027951)
(1600, 0.001, 1.158831, 1.1043295)
(1700, 0.001, 1.05643, 1.0890219)
(1800, 0.001, 1.2656322, 1.0992106)
(1900, 0.001, 0.77994013, 1.1061394)
(2000, 0.001, 1.0865188, 1.0994133)
(2100, 0.001, 1.0369456, 1.0963727)
(2200, 0.001, 0.95835686, 1.1430421)
(2300, 0.001, 1.0899954, 1.1166598)
(2400, 0.001, 1.3044115, 1.0928384)
(2500, 0.001, 1.0909076, 1.1153448)
(2600, 0.001, 0.90414369, 1.0940207)
(2700, 0.001, 0.95731258, 1.1155005)
(2800, 0.001, 1.0438951, 1.0716981)

In [13]:
loss_cumm=[]
lr=0.001
for t in range(50000):
    loss_t = train_batch(batch_size, lr=lr)
    loss_cumm += [loss_t]
    if t>=100 and t%100==0:
        print(t, lr, loss_t, np.mean(loss_cumm[-100:]))
    #Update lr if not improvements in last 50 batches
    #if t>100 and np.mean(loss_cumm[t-100:t-50])<np.mean(loss_cumm[t-50:]):  
    #    lr = lr*0.9999

(100, 0.001, 1.1623048, 1.0297424)
(200, 0.001, 1.0586894, 1.0282066)
(300, 0.001, 0.97221565, 1.0323272)
(400, 0.001, 0.62024659, 0.98647493)
(500, 0.001, 1.2645192, 1.0621926)
(600, 0.001, 0.98671681, 1.0183251)
(700, 0.001, 0.8942802, 1.0039647)
(800, 0.001, 0.79047954, 1.0257635)
(900, 0.001, 1.3309519, 1.0637367)
(1000, 0.001, 0.66823, 1.0163255)
(1100, 0.001, 0.87605119, 1.0566812)
(1200, 0.001, 0.87455595, 1.0540406)
(1300, 0.001, 0.64425439, 1.0569378)
(1400, 0.001, 1.3997583, 1.0509285)
(1500, 0.001, 1.0748911, 0.98795348)
(1600, 0.001, 0.85577297, 1.048548)
(1700, 0.001, 0.95112544, 1.0546268)
(1800, 0.001, 0.89367497, 1.0623887)
(1900, 0.001, 0.9690845, 1.0458704)
(2000, 0.001, 1.0375814, 1.043658)
(2100, 0.001, 0.96416634, 1.0245025)
(2200, 0.001, 1.1977797, 1.0162425)
(2300, 0.001, 1.0509284, 1.0478959)
(2400, 0.001, 0.94518119, 1.066987)
(2500, 0.001, 0.78589004, 1.0417681)
(2600, 0.001, 0.84294593, 1.0383606)
(2700, 0.001, 1.0863942, 1.0152516)
(2800, 0.001, 0.68129343, 

## Score new sentences

In [25]:
# Decode sampled_loss
output_projection = None


In [28]:

output_projection = (w_proy, b_proy)

In [23]:
from nltk import word_tokenize

#seq_length = 20
#vocab_size = 50000

PAD_ID = 0
UNK_ID = 3


In [24]:



#Input
init_pos = random.randint(0,len(input_sent_trn)-batch_size)
encoder_input = input_sent_trn[init_pos:init_pos+batch_size]
decoder_input = [[GO_ID] + [PAD_ID]*(seq_length + 1)]*batch_size#length = seq_length+2
encoder_input = np.array(encoder_input).T
decoder_input = np.array(decoder_input).T


# Prediction
feed_dict = {feed_previous: True} # False feed_previous in the trainig process.
feed_dict.update({enc_inp[t]: encoder_input[t] for t in range(seq_length)})
feed_dict.update({dec_inp[t]: decoder_input[t] for t in range(seq_length+2)})
response_predict = np.array(sess.run(outputs, feed_dict) ) 
print(response_predict.shape)
response_predict_text = decode_response(response_predict)

#Real
response_real = target_sent_trn[init_pos:init_pos+batch_size]
response_real_text = [' '.join([inv_worddict[r] for r in rr]) for rr in response_real]
encoder_input = input_sent_trn[init_pos:init_pos+batch_size]
input_real_text = [' '.join([inv_worddict[r] for r in rr]) for rr in encoder_input]

#Print 
for i in range(len(response_real_text)):
    print(input_real_text[i], ' - ', response_real_text[i], ' - ',response_predict_text[i])

(7, 32, 1024)
('PAD PAD PAD PAD UNK', ' - ', u'GO no podr\xe9 ver nada EOL PAD', ' - ', u'comer comer juego juego peor peor peor')
('PAD PAD PAD PAD UNK', ' - ', u'GO no s\xe9 cu\xe1l es EOL PAD', ' - ', u'comer comer juego juego peor peor peor')
(u'PAD no s\xe9 cu\xe1l es', ' - ', u'GO creo que es esta EOL PAD', ' - ', u'jimmy comer comer comer peor peor peor')
(u'PAD tiene un \xfaltimo intento', ' - ', u'GO aqu\xed est\xe1 EOL PAD PAD PAD', ' - ', u'para norte juego juego norte norte norte')
(u'PAD PAD \xa1 pap\xe1 !', ' - ', u'GO \xa1 UNK ! EOL PAD PAD', ' - ', u'para otro ventana para para peor peor')
(u'PAD PAD \xa1 UNK !', ' - ', u'GO \xa1 ay\xfadenme ! EOL PAD PAD', ' - ', u'para para ventana norte peor peor peor')
(u'PAD PAD \xa1 ay\xfadenme !', ' - ', u'GO \xa1 auxilio ! EOL PAD PAD', ' - ', u'para otro ventana para peor peor peor')
(u'PAD su problema esta lejos', ' - ', u'GO es un buen trabajador EOL PAD', ' - ', u'comer d\xedas comer comer norte peor peor')
(u'PAD es un buen

In [26]:

        
sentence = 'tiene mujer'

tokens = word_tokenize(sentence.decode('utf-8'), language='spanish')
print(tokens)

encoder_input = [PAD_ID]*(seq_length - len(tokens)) + [worddict_id(w) for w in tokens]
decoder_input = [GO_ID] + [PAD_ID]*(seq_length + 1) #length = seq_length+2

encoder_input = np.array([encoder_input]).T
decoder_input = np.array([decoder_input]).T

feed_dict = {feed_previous: True} # False feed_previous in the trainig process.
feed_dict.update({enc_inp[t]: encoder_input[t] for t in range(seq_length)})
feed_dict.update({dec_inp[t]: decoder_input[t] for t in range(seq_length+2)})
response = sess.run(dec_outputs, feed_dict)

print(len(response))
print(response[0].shape)


[u'tiene', u'mujer']
7
(1, 1024)


In [29]:
response_array = np.array(response)
print(response_array[:,0,4:].shape)
print(response_array[0,0,:20])
print(response_array[1,0,:20])

(7, 1020)
[-0.12829274 -0.66459137  0.01443711  0.09059531 -0.01850128  0.01826361
 -0.04814027  0.03603132 -0.1919519   0.11358448  0.09021166  0.08910131
 -0.03740841  0.13902247 -0.01688106 -0.08737825 -0.07229998 -0.09079627
  0.09607144  0.0753211 ]
[-0.21056122 -1.1050328   0.03634074  0.11899484 -0.05985006 -0.01769391
  0.00899929  0.05349598 -0.00759614  0.17625991  0.18409103  0.22430681
 -0.07216804  0.14274521  0.01827745 -0.0841601  -0.08550423 -0.08624136
  0.08744537  0.05118456]


In [ ]:
[-0.12829274 -0.66459137  0.01443711  0.09059531 -0.01850128  0.01826361
 -0.04814027  0.03603132 -0.1919519   0.11358448  0.09021166  0.08910131
 -0.03740841  0.13902247 -0.01688106 -0.08737825 -0.07229998 -0.09079627
  0.09607144  0.0753211 ]
[-0.21056122 -1.1050328   0.03634074  0.11899484 -0.05985006 -0.01769391
  0.00899929  0.05349598 -0.00759614  0.17625991  0.18409103  0.22430681
 -0.07216804  0.14274521  0.01827745 -0.0841601  -0.08550423 -0.08624136
  0.08744537  0.05118456]

In [18]:
for i in range(response_array.shape[1]):
    response_dec = [np.argmax(r) for r in response_array[:,i,:]]
    #print(response_dec)
    print(' '.join([inv_worddict[r] for r in response_dec]))



comer jimmy juego esposo peor peor peor


In [20]:
print(response[2][0][430:450])
print(np.argmax(response[2][0][:]))

[ 2.25358653 -1.59977579  0.90094829 -1.55935824 -0.66728562  1.16686046
 -0.96262938  2.57319927  0.44875813  0.65800786  3.81305313 -0.71065307
  1.06652832  0.53407711 -1.88079274  0.71391678  0.18740039 -0.3640964
  1.37924945 -1.52024591]
440


In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import random

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.models.rnn.translate import data_utils


class Seq2SeqModel(object):
  """Sequence-to-sequence model with attention and for multiple buckets.
  This class implements a multi-layer recurrent neural network as encoder,
  and an attention-based decoder. This is the same as the model described in
  this paper: http://arxiv.org/abs/1412.7449 - please look there for details,
  or into the seq2seq library for complete model implementation.
  This class also allows to use GRU cells in addition to LSTM cells, and
  sampled softmax to handle large output vocabulary size. A single-layer
  version of this model, but with bi-directional encoder, was presented in
    http://arxiv.org/abs/1409.0473
  and sampled softmax is described in Section 3 of the following paper.
    http://arxiv.org/abs/1412.2007
  """

  def __init__(self,
               source_vocab_size,
               target_vocab_size,
               buckets,
               size,
               num_layers,
               max_gradient_norm,
               batch_size,
               learning_rate,
               learning_rate_decay_factor,
               use_lstm=False,
               num_samples=512,
               forward_only=False,
               dtype=tf.float32):
    """Create the model.
    Args:
      source_vocab_size: size of the source vocabulary.
      target_vocab_size: size of the target vocabulary.
      buckets: a list of pairs (I, O), where I specifies maximum input length
        that will be processed in that bucket, and O specifies maximum output
        length. Training instances that have inputs longer than I or outputs
        longer than O will be pushed to the next bucket and padded accordingly.
        We assume that the list is sorted, e.g., [(2, 4), (8, 16)].
      size: number of units in each layer of the model.
      num_layers: number of layers in the model.
      max_gradient_norm: gradients will be clipped to maximally this norm.
      batch_size: the size of the batches used during training;
        the model construction is independent of batch_size, so it can be
        changed after initialization if this is convenient, e.g., for decoding.
      learning_rate: learning rate to start with.
      learning_rate_decay_factor: decay learning rate by this much when needed.
      use_lstm: if true, we use LSTM cells instead of GRU cells.
      num_samples: number of samples for sampled softmax.
      forward_only: if set, we do not construct the backward pass in the model.
      dtype: the data type to use to store internal variables.
    """
    self.source_vocab_size = source_vocab_size
    self.target_vocab_size = target_vocab_size
    self.buckets = buckets
    self.batch_size = batch_size
    self.learning_rate = tf.Variable(
        float(learning_rate), trainable=False, dtype=dtype)
    self.learning_rate_decay_op = self.learning_rate.assign(
        self.learning_rate * learning_rate_decay_factor)
    self.global_step = tf.Variable(0, trainable=False)

    # If we use sampled softmax, we need an output projection.
    output_projection = None
    softmax_loss_function = None
    # Sampled softmax only makes sense if we sample less than vocabulary size.
    if num_samples > 0 and num_samples < self.target_vocab_size:
      w_t = tf.get_variable("proj_w", [self.target_vocab_size, size], dtype=dtype)
      w = tf.transpose(w_t)
      b = tf.get_variable("proj_b", [self.target_vocab_size], dtype=dtype)
      output_projection = (w, b)

      def sampled_loss(inputs, labels):
        labels = tf.reshape(labels, [-1, 1])
        # We need to compute the sampled_softmax_loss using 32bit floats to
        # avoid numerical instabilities.
        local_w_t = tf.cast(w_t, tf.float32)
        local_b = tf.cast(b, tf.float32)
        local_inputs = tf.cast(inputs, tf.float32)
        return tf.cast(
            tf.nn.sampled_softmax_loss(local_w_t, local_b, local_inputs, labels,
                                       num_samples, self.target_vocab_size),
            dtype)
      softmax_loss_function = sampled_loss

    # Create the internal multi-layer cell for our RNN.
    single_cell = tf.nn.rnn_cell.GRUCell(size)
    if use_lstm:
      single_cell = tf.nn.rnn_cell.BasicLSTMCell(size)
    cell = single_cell
    if num_layers > 1:
      cell = tf.nn.rnn_cell.MultiRNNCell([single_cell] * num_layers)

    # The seq2seq function: we use embedding for the input and attention.
    def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
      return tf.nn.seq2seq.embedding_attention_seq2seq(
          encoder_inputs,
          decoder_inputs,
          cell,
          num_encoder_symbols=source_vocab_size,
          num_decoder_symbols=target_vocab_size,
          embedding_size=size,
          output_projection=output_projection,
          feed_previous=do_decode,
          dtype=dtype)

    # Feeds for inputs.
    self.encoder_inputs = []
    self.decoder_inputs = []
    self.target_weights = []
    for i in xrange(buckets[-1][0]):  # Last bucket is the biggest one.
      self.encoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="encoder{0}".format(i)))
    for i in xrange(buckets[-1][1] + 1):
      self.decoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="decoder{0}".format(i)))
      self.target_weights.append(tf.placeholder(dtype, shape=[None],
                                                name="weight{0}".format(i)))

    # Our targets are decoder inputs shifted by one.
    targets = [self.decoder_inputs[i + 1]
               for i in xrange(len(self.decoder_inputs) - 1)]

    # Training outputs and losses.
    if forward_only:
      self.outputs, self.losses = tf.nn.seq2seq.model_with_buckets(
          self.encoder_inputs, self.decoder_inputs, targets,
          self.target_weights, buckets, lambda x, y: seq2seq_f(x, y, True),
          softmax_loss_function=softmax_loss_function)
      # If we use output projection, we need to project outputs for decoding.
      if output_projection is not None:
        for b in xrange(len(buckets)):
          self.outputs[b] = [
              tf.matmul(output, output_projection[0]) + output_projection[1]
              for output in self.outputs[b]
          ]
    else:
      self.outputs, self.losses = tf.nn.seq2seq.model_with_buckets(
          self.encoder_inputs, self.decoder_inputs, targets,
          self.target_weights, buckets,
          lambda x, y: seq2seq_f(x, y, False),
          softmax_loss_function=softmax_loss_function)

    # Gradients and SGD update operation for training the model.
    params = tf.trainable_variables()
    if not forward_only:
      self.gradient_norms = []
      self.updates = []
      opt = tf.train.GradientDescentOptimizer(self.learning_rate)
      for b in xrange(len(buckets)):
        gradients = tf.gradients(self.losses[b], params)
        clipped_gradients, norm = tf.clip_by_global_norm(gradients,
                                                         max_gradient_norm)
        self.gradient_norms.append(norm)
        self.updates.append(opt.apply_gradients(
            zip(clipped_gradients, params), global_step=self.global_step))

    self.saver = tf.train.Saver(tf.all_variables())

  def step(self, session, encoder_inputs, decoder_inputs, target_weights,
           bucket_id, forward_only):
    """Run a step of the model feeding the given inputs.
    Args:
      session: tensorflow session to use.
      encoder_inputs: list of numpy int vectors to feed as encoder inputs.
      decoder_inputs: list of numpy int vectors to feed as decoder inputs.
      target_weights: list of numpy float vectors to feed as target weights.
      bucket_id: which bucket of the model to use.
      forward_only: whether to do the backward step or only forward.
    Returns:
      A triple consisting of gradient norm (or None if we did not do backward),
      average perplexity, and the outputs.
    Raises:
      ValueError: if length of encoder_inputs, decoder_inputs, or
        target_weights disagrees with bucket size for the specified bucket_id.
    """
    # Check if the sizes match.
    encoder_size, decoder_size = self.buckets[bucket_id]
    if len(encoder_inputs) != encoder_size:
      raise ValueError("Encoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(encoder_inputs), encoder_size))
    if len(decoder_inputs) != decoder_size:
      raise ValueError("Decoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(decoder_inputs), decoder_size))
    if len(target_weights) != decoder_size:
      raise ValueError("Weights length must be equal to the one in bucket,"
                       " %d != %d." % (len(target_weights), decoder_size))

    # Input feed: encoder inputs, decoder inputs, target_weights, as provided.
    input_feed = {}
    for l in xrange(encoder_size):
      input_feed[self.encoder_inputs[l].name] = encoder_inputs[l]
    for l in xrange(decoder_size):
      input_feed[self.decoder_inputs[l].name] = decoder_inputs[l]
      input_feed[self.target_weights[l].name] = target_weights[l]

    # Since our targets are decoder inputs shifted by one, we need one more.
    last_target = self.decoder_inputs[decoder_size].name
    input_feed[last_target] = np.zeros([self.batch_size], dtype=np.int32)

    # Output feed: depends on whether we do a backward step or not.
    if not forward_only:
      output_feed = [self.updates[bucket_id],  # Update Op that does SGD.
                     self.gradient_norms[bucket_id],  # Gradient norm.
                     self.losses[bucket_id]]  # Loss for this batch.
    else:
      output_feed = [self.losses[bucket_id]]  # Loss for this batch.
      for l in xrange(decoder_size):  # Output logits.
        output_feed.append(self.outputs[bucket_id][l])

    outputs = session.run(output_feed, input_feed)
    if not forward_only:
      return outputs[1], outputs[2], None  # Gradient norm, loss, no outputs.
    else:
      return None, outputs[0], outputs[1:]  # No gradient norm, loss, outputs.

  def get_batch(self, data, bucket_id):
    """Get a random batch of data from the specified bucket, prepare for step.
    To feed data in step(..) it must be a list of batch-major vectors, while
    data here contains single length-major cases. So the main logic of this
    function is to re-index data cases to be in the proper format for feeding.
    Args:
      data: a tuple of size len(self.buckets) in which each element contains
        lists of pairs of input and output data that we use to create a batch.
      bucket_id: integer, which bucket to get the batch for.
    Returns:
      The triple (encoder_inputs, decoder_inputs, target_weights) for
      the constructed batch that has the proper format to call step(...) later.
    """
    encoder_size, decoder_size = self.buckets[bucket_id]
    encoder_inputs, decoder_inputs = [], []

    # Get a random batch of encoder and decoder inputs from data,
    # pad them if needed, reverse encoder inputs and add GO to decoder.
    for _ in xrange(self.batch_size):
      encoder_input, decoder_input = random.choice(data[bucket_id])

      # Encoder inputs are padded and then reversed.
      encoder_pad = [data_utils.PAD_ID] * (encoder_size - len(encoder_input))
      encoder_inputs.append(list(reversed(encoder_input + encoder_pad)))

      # Decoder inputs get an extra "GO" symbol, and are padded then.
      decoder_pad_size = decoder_size - len(decoder_input) - 1
      decoder_inputs.append([data_utils.GO_ID] + decoder_input +
                            [data_utils.PAD_ID] * decoder_pad_size)

    # Now we create batch-major vectors from the data selected above.
    batch_encoder_inputs, batch_decoder_inputs, batch_weights = [], [], []

    # Batch encoder inputs are just re-indexed encoder_inputs.
    for length_idx in xrange(encoder_size):
      batch_encoder_inputs.append(
          np.array([encoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

    # Batch decoder inputs are re-indexed decoder_inputs, we create weights.
    for length_idx in xrange(decoder_size):
      batch_decoder_inputs.append(
          np.array([decoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

      # Create target_weights to be 0 for targets that are padding.
      batch_weight = np.ones(self.batch_size, dtype=np.float32)
      for batch_idx in xrange(self.batch_size):
        # We set weight to 0 if the corresponding target is a PAD symbol.
        # The corresponding target is decoder_input shifted by 1 forward.
        if length_idx < decoder_size - 1:
          target = decoder_inputs[batch_idx][length_idx + 1]
        if length_idx == decoder_size - 1 or target == data_utils.PAD_ID:
          batch_weight[batch_idx] = 0.0
      batch_weights.append(batch_weight)
    return batch_encoder_inputs, batch_decoder_inputs, batch_weights

In [3]:
def self_test():
  """Test the translation model."""
  with tf.Session() as sess:
    print("Self-test for neural translation model.")
    # Create model with vocabularies of 10, 2 small buckets, 2 layers of 32, batch of 32.
    model = seq2seq_model.Seq2SeqModel(10, 10, [(3, 3), (6, 6)], 32, 2,
                                       5.0, 32, 0.3, 0.99, num_samples=8)

    
    sess.run(tf.initialize_all_variables())

    # Fake data set for both the (3, 3) and (6, 6) bucket.
    data_set = ([([1, 1], [2, 2]), ([3, 3], [4]), ([5], [6])],
                [([1, 1, 1, 1, 1], [2, 2, 2, 2, 2]), ([3, 3, 3], [5, 6])])
    for _ in xrange(5):  # Train the fake model for 5 steps.
      bucket_id = random.choice([0, 1])
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          data_set, bucket_id)
      print('encoder_inputs:',encoder_inputs)
      print('decoder_inputs:',decoder_inputs)
      print('target_weights:',target_weights)
      o1, o2, o3 = model.step(sess, encoder_inputs, decoder_inputs, target_weights,
                 bucket_id, False)
      print('ooo:',o1,o2,o3)
    print('Done!')

In [4]:
from tensorflow.models.rnn.translate import seq2seq_model
self_test()

Self-test for neural translation model.
encoder_inputs: [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), array([0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0], dtype=int32), array([0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0], dtype=int32), array([3, 3, 1, 1, 1, 3, 3, 3, 1, 1, 3, 3, 1, 3, 1, 3, 3, 1, 1, 1, 3, 1, 1,
       1, 3, 3, 1, 3, 3, 1, 3, 3], dtype=int32), array([3, 3, 1, 1, 1, 3, 3, 3, 1, 1, 3, 3, 1, 3, 1, 3, 3, 1, 1, 1, 3, 1, 1,
       1, 3, 3, 1, 3, 3, 1, 3, 3], dtype=int32), array([3, 3, 1, 1, 1, 3, 3, 3, 1, 1, 3, 3, 1, 3, 1, 3, 3, 1, 1, 1, 3, 1, 1,
       1, 3, 3, 1, 3, 3, 1, 3, 3], dtype=int32)]
decoder_inputs: [array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32), array([5, 5, 2, 2, 2, 5, 5, 5, 2, 2, 5, 5, 2, 5, 2,

In [ ]:



tf.nn.seq2seq.embedding_rnn_seq2seq(encoder_inputs, decoder_inputs, cell, 
                                    num_encoder_symbols, num_decoder_symbols,
                                    embedding_size)


tf.nn.seq2seq.model_with_buckets()

# TEst of seq2seq simple

In [10]:
vocab_size=10
seq_length=3
batch_size=32
X = [np.random.choice(vocab_size, size=(seq_length,), replace=False)
         for _ in range(batch_size)]
print(X)
Y = X[:]
Y = [np.concatenate((np.zeros_like(x[:-1]), [np.sum(x)])) for x in X]
Y = [[xi+i for i, xi in enumerate(x)] for x in X]
print(Y)
    # Dimshuffle to seq_len * batch_size
X = np.array(X).T
Y = np.array(Y).T

print(X)
print(Y)

[array([5, 1, 0]), array([0, 7, 4]), array([2, 7, 9]), array([8, 1, 3]), array([0, 2, 9]), array([4, 9, 2]), array([5, 7, 2]), array([9, 8, 7]), array([5, 0, 2]), array([8, 6, 1]), array([8, 5, 0]), array([3, 0, 5]), array([6, 0, 8]), array([4, 6, 0]), array([2, 3, 8]), array([1, 2, 4]), array([1, 5, 7]), array([4, 9, 8]), array([3, 6, 1]), array([3, 8, 9]), array([6, 2, 8]), array([5, 8, 0]), array([0, 8, 1]), array([2, 4, 7]), array([9, 8, 7]), array([6, 0, 4]), array([4, 0, 5]), array([4, 9, 8]), array([2, 3, 8]), array([0, 9, 1]), array([9, 7, 3]), array([4, 2, 3])]
[[5, 2, 2], [0, 8, 6], [2, 8, 11], [8, 2, 5], [0, 3, 11], [4, 10, 4], [5, 8, 4], [9, 9, 9], [5, 1, 4], [8, 7, 3], [8, 6, 2], [3, 1, 7], [6, 1, 10], [4, 7, 2], [2, 4, 10], [1, 3, 6], [1, 6, 9], [4, 10, 10], [3, 7, 3], [3, 9, 11], [6, 3, 10], [5, 9, 2], [0, 9, 3], [2, 5, 9], [9, 9, 9], [6, 1, 6], [4, 1, 7], [4, 10, 10], [2, 4, 10], [0, 10, 3], [9, 8, 5], [4, 3, 5]]
[[5 0 2 8 0 4 5 9 5 8 8 3 6 4 2 1 1 4 3 3 6 5 0 2 9 6 4 4

In [1]:
import tensorflow as tf
import numpy as np


#Start interactive session
#sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
sess = tf.InteractiveSession()

seq_length = 5
batch_size = 32

vocab_size = 7
embedding_dim = 50

memory_dim = 300


enc_inp = [tf.placeholder(tf.int32, shape=(None,),
                          name="inp%i" % t)
           for t in range(seq_length)]

labels = [tf.placeholder(tf.int32, shape=(None,),
                        name="labels%i" % t)
          for t in range(seq_length)]

weights = [tf.ones_like(labels_t, dtype=tf.float32)
           for labels_t in labels]

# Decoder input: prepend some "GO" token and drop the final
# token of the encoder input
dec_inp = ([tf.zeros_like(enc_inp[0], dtype=np.int32, name="GO")]
           + enc_inp[:-1])
print(dec_inp)

# Initial memory value for recurrence.
prev_mem = tf.zeros((batch_size, memory_dim))


cell = tf.nn.rnn_cell.GRUCell(memory_dim)

dec_outputs, dec_memory = tf.nn.seq2seq.embedding_rnn_seq2seq(
    enc_inp, dec_inp, cell, vocab_size, vocab_size+5, embedding_dim)
print(dec_outputs)


loss = tf.nn.seq2seq.sequence_loss(dec_outputs, labels, weights, vocab_size)
#loss = tf.nn.log_softmax()

learning_rate = 0.01
momentum = 0.9
optimizer = tf.train.AdamOptimizer(learning_rate)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)



[<tf.Tensor 'GO:0' shape=(?,) dtype=int32>, <tf.Tensor 'inp0:0' shape=(?,) dtype=int32>, <tf.Tensor 'inp1:0' shape=(?,) dtype=int32>, <tf.Tensor 'inp2:0' shape=(?,) dtype=int32>, <tf.Tensor 'inp3:0' shape=(?,) dtype=int32>]
[<tf.Tensor 'embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/OutputProjectionWrapper/add:0' shape=(?, 12) dtype=float32>, <tf.Tensor 'embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/OutputProjectionWrapper_1/add:0' shape=(?, 12) dtype=float32>, <tf.Tensor 'embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/OutputProjectionWrapper_2/add:0' shape=(?, 12) dtype=float32>, <tf.Tensor 'embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/OutputProjectionWrapper_3/add:0' shape=(?, 12) dtype=float32>, <tf.Tensor 'embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/OutputProjectionWrapper_4/add:0' shape=(?, 12) dtype=float32>]


In [2]:
sess.run(tf.initialize_all_variables())

In [4]:
def train_batch(batch_size):
    X = [np.random.choice(vocab_size, size=(seq_length,), replace=False)
         for _ in range(batch_size)]
    #Y = X[:]
    # Sum calculator
    Y = [[xi+i for i, xi in enumerate(x)] for x in X]
    
    # Dimshuffle to seq_len * batch_size
    X = np.array(X).T
    Y = np.array(Y).T

    feed_dict = {enc_inp[t]: X[t] for t in range(seq_length)}
    feed_dict.update({labels[t]: Y[t] for t in range(seq_length)})

    _, loss_t = sess.run([train_op, loss], feed_dict)
    return loss_t

for t in range(500):
    loss_t = train_batch(batch_size)
    if t%100==0:
        print(t, loss_t)


(0, 2.4965401)
(100, 0.071821749)
(200, 0.022222163)
(300, 0.026851129)
(400, 0.00049195113)


In [5]:
X_batch = [np.random.choice(vocab_size, size=(seq_length,), replace=False)
           for _ in range(10)]
X_batch = np.array(X_batch).T

feed_dict = {enc_inp[t]: X_batch[t] for t in range(seq_length)}
dec_outputs_batch = sess.run(dec_outputs, feed_dict)

print(X_batch)

print([logits_t.argmax(axis=1) for logits_t in dec_outputs_batch])

In [ ]:
#use of embedings pretrained

# load word2vec model (say from gensim)
model = load_model(FILENAME, binary=True)

# embedding matrix
X = model.syn0
print(type(X)) # numpy.ndarray
print(X.shape) # (vocab_size, embedding_dim)

# start interactive session
sess = tf.InteractiveSession()

# set embeddings
embeddings = tf.Variable(tf.random_uniform(X.shape, minval=-0.1, maxval=0.1), trainable=False)

# initialize
sess.run(tf.initialize_all_variables())

# override inits
sess.run(embeddings.assign(X))




## 03 Test model